In [ ]:
import mediapipe as mp
import time
import keyboard
import pyautogui
import playsound
import cv2
import pygame
import threading

In [ ]:


class handDetector:
    def __init__(self, mode=False, maxHands=1, detectionCon=0.5, trackCon=0.5, model_complexity=1):
        self.last_gesture = None
        self.last_arrow_time = time.time()
        self.last_thumbs_time = time.time()
        self.last_index_time = time.time()
        self.last_space_time = time.time()
        self.last_fist_time = time.time()
        self.spacebar_allowed = True
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.model_complexity = model_complexity
        self.mp_hands = mp.solutions.hands.Hands(
            self.mode, self.maxHands,
            model_complexity=self.model_complexity,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.trackCon
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.results = None

    def findHands(self, image, draw=True):
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.mp_hands.process(imageRGB)
        if self.results.multi_hand_landmarks and draw:
            for handLms in self.results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(image, handLms, mp.solutions.hands.HAND_CONNECTIONS)
        return image

    def findPos(self, image, handNumber=0, draw=True):
        
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNumber]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(image, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
            print("lmList:", lmList)  # Add print statements for debugging
            print("Multi-hand landmarks:", self.results.multi_hand_landmarks)

            gesture = self.detectGesture(lmList)
            cv2.putText(image, gesture, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        return lmList
    
    def detectGesture(self, lmList, prev_lmList=[], hand_side="Unknown"):
        # If no landmarks are detected, return unknown
        if len(lmList) == 0:
            return "Unknown"
        
        x1, x2 = lmList[8][1], lmList[12][1]

        # Get the tip landmarks (ids: 4, 8, 12, 16, 20) and middle landmarks (ids: 3, 7, 11, 15, 19)
        tips = [lmList[i][2] for i in [4, 8, 12, 16, 20]]
        middles = [lmList[i][2] for i in [3, 7, 11, 15, 19]]
        

    # Otherwise, return unknown

        # Get the y-coordinates of the tips, knuckles, and middle fingers
        tips_y = [lmList[i][2] for i in [4, 8, 12, 16, 20]]
        knuckle_1_y = lmList[0][2]  # Base of Thumb
        knuckle_2_y = lmList[1][2]  # Knuckle for thumb
        knuckle_3_y = lmList[2][2]  # Pivot of thumb
        knuckle_4_y = lmList[3][2]  # Tip of thumb
        
        knuckle_5_y = lmList[4][2]  # Knuckle for index finger
        knuckle_6_y = lmList[5][2]  # 1st Pivot for index finger
        knuckle_7_y = lmList[6][2]  # 2nd Pivot for index finger
        knuckle_8_y = lmList[7][2] # Tip for index finger
        
        knuckle_9_y = lmList[8][2]  # Knuckle for middle finger
        knuckle_10_y = lmList[9][2]  # 1st Pivot for middle finger
        knuckle_11_y = lmList[10][2]  # 2nd Pivot for middle finger
        knuckle_12_y = lmList[11][2]    # Tip for middle finger
        
        knuckle_13_y = lmList[12][2]  # Knuckle for ring finger
        knuckle_14_y = lmList[13][2]  # 1st Pivot for ring finger
        knuckle_15_y = lmList[14][2]  # 2nd Pivot for ring finger
        knuckle_16_y = lmList[15][2]  # Tip for ring finger
        
        knuckle_17_y = lmList[16][2]  # Knuckle for pinky finger
        knuckle_18_y = lmList[17][2]  # 1st Pivot for pinky finger
        knuckle_19_y = lmList[18][2]  # 2nd Pivot for pinky finger
        knuckle_20_y = lmList[19][2]  # Tip for pinky finger
        
        knuckle_21_y = lmList[20][2] #base of the palm

        # Calculate the vertical distances between the tips and the knuckles
        distance_knuckle_6 = abs(tips_y[0] - knuckle_6_y)
        distance_knuckle_7 = abs(tips_y[1] - knuckle_7_y)
        distance_knuckle_10 = abs(tips_y[2] - knuckle_10_y)
        distance_knuckle_11 = abs(tips_y[3] - knuckle_11_y)
        distance_knuckle_14 = abs(tips_y[2] - knuckle_14_y)
        distance_knuckle_15 = abs(tips_y[3] - knuckle_15_y)
        distance_knuckle_18 = abs(tips_y[2] - knuckle_18_y)
        distance_knuckle_19 = abs(tips_y[3] - knuckle_19_y)

        # Define thresholds for distinguishing between gestures
        fist_threshold = 10  # Adjust this threshold based on your observations
        open_palm_threshold = 20  # Adjust this threshold based on your observations
        two_fingers_threshold = 30  # Adjust this threshold based on your observations
        
        if all(tip_y - knuckle_y < open_palm_threshold for tip_y, knuckle_y in zip(tips_y, [knuckle_6_y, knuckle_7_y, knuckle_10_y, knuckle_11_y, knuckle_11_y])):
            return "Open Palm"
        
        elif all(tip_y - knuckle_y > fist_threshold for tip_y, knuckle_y in zip(tips_y, [knuckle_6_y, knuckle_7_y, knuckle_10_y, knuckle_11_y, knuckle_11_y])):
            return "Fist"
        
        
        elif (tips_y[1] < knuckle_7_y and tips_y[2] < knuckle_11_y and tips_y[3] < knuckle_15_y and tips_y[4] < knuckle_19_y):
            return "Four Fingers"
            
            # If there are previous landmarks, compare the current x-coordinates with the previous ones
            #elif prev_lmList:
                
            prev_x1, prev_x2 = prev_lmList[8][1], prev_lmList[12][1]
            if x1 < prev_x1 and x2 < prev_x2:
                return "Left Swipe"
            elif x1 > prev_x1 and x2 > prev_x2:
                return "Right Swipe"
            
            
        elif ((tips_y[1] < knuckle_7_y and tips_y[2] < knuckle_11_y) and (tips_y[3] > knuckle_14_y and tips_y[4] > knuckle_18_y)):
            return "Two Fingers"
        
        elif (((knuckle_3_y) < knuckle_6_y and knuckle_10_y and knuckle_14_y and knuckle_18_y)) and ((knuckle_2_y) < (knuckle_6_y and knuckle_10_y and knuckle_14_y and knuckle_18_y)):  
            return "Thumbs Up"
        
        elif (((tips_y[4]<knuckle_5_y) and knuckle_1_y<knuckle_3_y) and ((knuckle_2_y) > (knuckle_6_y and knuckle_10_y and knuckle_14_y and knuckle_18_y) and (knuckle_3_y) > (knuckle_6_y and knuckle_10_y and knuckle_14_y and knuckle_18_y))):  
            return "Thumbs Down"
        
        elif ((((knuckle_6_y and knuckle_7_y) < (knuckle_10_y and knuckle_11_y) and (knuckle_14_y and knuckle_15_y) and (knuckle_18_y and knuckle_19_y)) and (knuckle_6_y and knuckle_7_y and tips_y[1])< knuckle_5_y)):
            return "Index Finger"
        
        elif (((tips_y[2]<(knuckle_9_y and knuckle_10_y) and (tips_y[1] and tips_y[3] and tips_y[4])>knuckle_9_y))):
            return "Middle Finger"


            # FOR RIGHT HAND
            if hand_side == "Right":
                if gesture=="Fist":
                    return "Right Fist"

            # FOR LEFT HAND
            elif hand_side == "Left":
                if gesture == "Two Fingers":
                    return "Left Two Fingers"
                


        # If none of the above conditions are met, return unknown
        else:
            return "Unknown"


    
    

In [ ]:
def main():
    pTime = 0
    cTime = 0
    pygame.mixer.init()
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    

    audio_file = "D:\Vstudio\Vine Boom Sound Effect (Bass Boosted).mp3"
    audio_file2 = "D:\Vstudio\Fart Meme Sound.mp3"
    while True:
        success, image = cap.read()
        image = detector.findHands(image)
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        image = cv2.flip(image, 1)
        cv2.putText(
            image,
            str(int(fps)),
            (500,400),
        cv2.FONT_HERSHEY_PLAIN,
        3,
        (120, 52, 40),
        3,
        )
        
        landmarks = detector.findPos(image)
        cv2.imshow("Camera result", image)
        if len(landmarks) != 0:
            print(landmarks[4])
         # Check for the 'Esc' key (27 in ASCII) to exit the program
        key = cv2.waitKey(1)
        if key == 27:  # 'Esc' key
            break
        
        
         # Process gestures
        gesture = detector.detectGesture(landmarks) 
        if gesture == "Open Palm" and time.time() - detector.last_space_time >= 2:
            keyboard.press_and_release('space')#Play/Pause
            detector.last_space_time = time.time()
        
        elif gesture == "Fist" and time.time() - detector.last_fist_time >= 1:
            pygame.mixer.music.load(audio_file)
            pygame.mixer.music.play()
            time.sleep(2)
        
        elif gesture == "Middle Finger" and time.time() - detector.last_fist_time >= 1:
            pygame.mixer.music.load(audio_file2)
            pygame.mixer.music.play()
            time.sleep(2)
        #if gesture == "Left Swipe":
            # Perform the action for the left swipe gesture
            #keyboard.press_and_release('left')
            #detector.last_arrow_time = time.time() 
        #elif gesture == "Right Swipe":
            # Perform the action for the right swipe gesture
            #keyboard.press_and_release('right')
            #detector.last_arrow_time = time.time() 
            
        elif gesture == "Left Two Fingers" and time.time() - detector.last_arrow_time >= 1:
            keyboard.press_and_release('right')  #5 seconds forward
            detector.last_arrow_time = time.time()

        elif gesture == "Four Fingers" and time.time() - detector.last_arrow_time >= 1:
            keyboard.press_and_release('left')  #5 seconds back
            detector.last_arrow_time = time.time() 
        
        elif gesture == "Thumbs Up" and time.time() - detector.last_thumbs_time >= 5 :
            keyboard.press_and_release('shift+n')  #Next Video
            detector.last_thumbs_time = time.time()
        
        elif gesture == "Thumbs Down" and time.time() - detector.last_thumbs_time >= 5:
            keyboard.press_and_release('alt+left')  #Previous Video
            detector.last_thumbs_time = time.time()
            
        elif gesture == "Index Finger" and time.time() - detector.last_index_time >= 3:
            keyboard.press_and_release('f') #Fullscreen
            detector.last_index_time = time.time()
    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()

        
if __name__ == "__main__":
    main() 